In [ ]:
# ----------- params ----------- #

writefiles = True
render_maps = False
perimeter_buffer = 15000 #TESTING switch to 15000 for prod

crs_global = 4326

In [6]:
# ----------- init -----------
import os
import glob
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
from datetime import date
import folium
from shapely.geometry import LineString, MultiLineString
import plotly.graph_objects as go
from collections import defaultdict
from datetime import datetime


In [7]:

# ----------- load ----------#
# Find all matching files
files = glob.glob("data/processed/network_*.gpkg")

# Get the most recent by modification time
gpkg_path = max(files, key=os.path.getmtime)

cycle_edges = gpd.read_file(gpkg_path, layer="edges_extended")
cycle_nodes = gpd.read_file(gpkg_path, layer="nodes")

DataLayerError: Layer 'edges_extended' could not be opened

## stats

In [ ]:
# ----------- share of assigned type masters by length -----------
lengthstat = cycle_edges[cycle_edges['reversed'] == False]
(lengthstat.groupby('cycleway_master')['length'].sum() / lengthstat['length'].sum()).round(3)

In [ ]:
# ----------- share matrix ----------- #
counts = cycle_edges.groupby(['oneway', 'oneway:bicycle'], dropna=False).size() .unstack(fill_value=0)
props = (counts / len(cycle_edges)).round(6)
props

In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['highway', 'cycleway_master'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['highway', 'cycleway_master'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['cycleway:right', 'cycleway_master'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['highway', 'cycleway:left'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['cycleway:left', 'oneway:bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['oneway', 'oneway:bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['highway', 'bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- count matrix ----------- #
cycle_edges.groupby(['cycleway', 'bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- diagnose "cycleway:right" vs "cycleway----------- #
cycle_edges.groupby(['cycleway:left', 'cycleway:right'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- diagnose ----------- #
cycle_edges.groupby(['highway', 'bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:
# ----------- diagnose ----------- #
cycle_edges.groupby(['highway', 'bicycle'], dropna=False).size().unstack(fill_value=0)


In [ ]:

df_plot = cycle_edges.copy()
#df_plot = df_plot[(df_plot['cycleway_master'] != "no") | (df_plot['highway'] == "cycleway")]

# ----------- Plot OSM comparison -----------

# Center map on average coordinate
center = df_plot['geometry'].union_all().centroid.coords[0]
m = folium.Map(location=[center[1], center[0]], zoom_start=12)

# OSM highway style mapping
style_map = {
    'primary':       {'color': 'red',    'weight': 4},
    'secondary':     {'color': 'blue',   'weight': 4},
    'tertiary':      {'color': 'yellow', 'weight': 4},
    'unclassified':  {'color': 'gray',   'weight': 3},
    'residential':   {'color': 'brown',  'weight': 3},
    'service':       {'color': 'black',  'weight': 3},
    'living_street': {'color': 'purple', 'weight': 2},
    'footway':       {'color': 'pink',   'weight': 2},
    'cycleway':      {'color': 'cyan',   'weight': 4}
}

for _, row in df_plot.iterrows():
    geom = row.geometry
    style = style_map.get(row['highway'], {'color': 'lightgray', 'weight': 1})

    tooltip = folium.Tooltip(
        f"u: {row.get('u', '')}<br>"
        f"v: {row.get('v', '')}<br>"
        f"highway: {row.get('highway', '')}<br>"
        f"cycleway: {row.get('cycleway', '')}<br>"
        f"cycleway:left: {row.get('cycleway:left', '')}<br>"
        f"cycleway:right: {row.get('cycleway:right', '')}<br>"
        f"cycleway:both: {row.get('cycleway:both', '')}<br>"
        f"cycleway_master: {row.get('cycleway_master', '')}"
    )

    if isinstance(geom, LineString):
        coords = [(pt[1], pt[0]) for pt in geom.coords]
        folium.PolyLine(
            locations=coords,
            color=style['color'],
            weight=style['weight'],
            opacity=0.6,
            tooltip=tooltip
        ).add_to(m)

    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(pt[1], pt[0]) for pt in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color=style['color'],
                weight=style['weight'],
                opacity=0.6,
                tooltip=tooltip
            ).add_to(m)

# Display map
display(m)
